In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import time
import random
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('stock_code.csv')
df

In [ ]:
df['stock_code']=df['stock_code'].str.slice(0,-3)

In [ ]:
stock_code=list(df['stock_code'])
stock_code

In [ ]:
annual_rep=pd.DataFrame()

driver = webdriver.Chrome()

#driver = webdriver.Chrome()
driver.implicitly_wait(604800)
new_list=[]
no_annual=[]

stock=['7148']
for s in stock_code:
    try:
        url='https://www.klsescreener.com/v2/stocks/view/'+s
        driver.get(url)
#         p= driver.find_element(By.XPATH,'//*[@id="myTab"]/li[3]/a')
#         driver.execute_script("arguments[0].click();",p)
        content = driver.page_source
        soup=bs(content)

        # scrap table

        tables = soup.find_all('table')
        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        i = 0
        has_found = False
        while i < len(dfs) and not has_found:
            n = 0
            while n < len(dfs[i].columns):
                if dfs[i].columns[n] == 'DP%':
                    has_found = True
                n += 1
            i += 1
        if has_found:
            df=dfs[i-1]
            df2=df[['Financial Year', "Revenue ('000)", "Net ('000)", 'EPS', 'DP%']]
            df2['Financial Year'] = pd.to_datetime(df['Financial Year'])
            df2['Financial Year']= pd.to_datetime(df2['Financial Year'].dt.strftime('%Y-%m-%d'))
            filtered_df = df2.loc[(df2['Financial Year'] >= '2018-01-01')
                          & (df2['Financial Year'] < '2023-12-31')]
            filtered_df.reset_index(drop=True, inplace=True)
#             filtered_df.drop(filtered_df[filtered_df['EPS'] == filtered_df['Revenue']].index, inplace = True)
#             filtered_df.reset_index(drop=True, inplace=True)
            filtered_df['Stock Code']=s
#             filtered_df['Year'] = filtered_df['Financial Year'].dt.strftime("%Y")
            annual_rep = annual_rep.append(filtered_df, ignore_index=True)
            print("Stock code :"+s+" yes")
        else:
            no_annual.append(s)
            print("Stock code :"+s+" has no annual report")
    except:
        print("Stock code :"+s+" cannot scrap")
    # Program to generate a random number between 5 to 30
    # importing the random module
    sec = random.randint(3, 10)

    # program scrapping waiting time
    time.sleep(sec)  # Seconds

In [ ]:
annual_rep

In [ ]:
annual_rep.to_csv("div_annual_report.csv")

In [2]:
df2 = pd.read_csv("div_annual_report.csv")

In [3]:
df2

,Unnamed: 0,Financial Year,Revenue ('000),Net ('000),EPS,DP%,Stock Code
0,0,2022-12-31,2771856,63803,5.66,-,5250
1,1,2021-12-31,2809085,43483,3.86,-,5250
2,2,2020-12-31,2537563,29767,2.60,-,5250
3,3,2019-12-31,2359399,54059,4.74,-,5250
4,4,2018-12-31,2216099,51307,4.56,-,5250
...,...,...,...,...,...,...,...
5212,5212,2022-11-30,136167,42487,8.33,2.04,5131
5213,5213,2021-11-30,149616,45360,9.86,1.72,5131
5214,5214,2020-11-30,170678,46773,10.17,1.67,5131
5215,5215,2019-11-30,170622,49522,10.77,1.11,5131


In [4]:
del df2['Unnamed: 0']

In [5]:
df2.dtypes

Financial Year     object
Revenue ('000)      int64
Net ('000)          int64
EPS               float64
DP%                object
Stock Code         object
dtype: object

In [6]:
df2.rename(columns={"Financial Year": "annual_financial_year", "Revenue ('000)": "annual_revenue('000)", "Net ('000)": "annual_net_income('000)", "EPS": "annual_eps", "DP%": "annual_dp(%)",
                    "Stock Code": "annual_stock_code"}, inplace=True)

In [7]:
df2

,annual_financial_year,annual_revenue('000),annual_net_income('000),annual_eps,annual_dp(%),annual_stock_code
0,2022-12-31,2771856,63803,5.66,-,5250
1,2021-12-31,2809085,43483,3.86,-,5250
2,2020-12-31,2537563,29767,2.60,-,5250
3,2019-12-31,2359399,54059,4.74,-,5250
4,2018-12-31,2216099,51307,4.56,-,5250
...,...,...,...,...,...,...
5212,2022-11-30,136167,42487,8.33,2.04,5131
5213,2021-11-30,149616,45360,9.86,1.72,5131
5214,2020-11-30,170678,46773,10.17,1.67,5131
5215,2019-11-30,170622,49522,10.77,1.11,5131


In [8]:
df2["annual_revenue('000)"] = df2["annual_revenue('000)"].apply(pd.to_numeric, errors='coerce')
df2["annual_net_income('000)"] = df2["annual_net_income('000)"].apply(pd.to_numeric, errors='coerce')
df2["annual_dp(%)"] = df2["annual_dp(%)"].apply(pd.to_numeric, errors='coerce')

df2['annual_financial_year'] = pd.to_datetime(df2['annual_financial_year'])
df2['annual_financial_year'] = pd.to_datetime(df2['annual_financial_year'].dt.strftime('%Y-%m-%d'))

In [9]:
df2['div_year'] = df2['annual_financial_year'].dt.strftime('%Y')

In [10]:
df2

,annual_financial_year,annual_revenue('000),annual_net_income('000),annual_eps,annual_dp(%),annual_stock_code,div_year
0,2022-12-31,2771856,63803,5.66,NaN,5250,2022
1,2021-12-31,2809085,43483,3.86,NaN,5250,2021
2,2020-12-31,2537563,29767,2.60,NaN,5250,2020
3,2019-12-31,2359399,54059,4.74,NaN,5250,2019
4,2018-12-31,2216099,51307,4.56,NaN,5250,2018
...,...,...,...,...,...,...,...
5212,2022-11-30,136167,42487,8.33,2.04,5131,2022
5213,2021-11-30,149616,45360,9.86,1.72,5131,2021
5214,2020-11-30,170678,46773,10.17,1.67,5131,2020
5215,2019-11-30,170622,49522,10.77,1.11,5131,2019


In [11]:
df2['annual_eps'] = df2['annual_eps']/100

In [13]:
df2['annual_eps']=df2['annual_eps'].round(3)

In [14]:
df2

,annual_financial_year,annual_revenue('000),annual_net_income('000),annual_eps,annual_dp(%),annual_stock_code,div_year
0,2022-12-31,2771856,63803,0.057,NaN,5250,2022
1,2021-12-31,2809085,43483,0.039,NaN,5250,2021
2,2020-12-31,2537563,29767,0.026,NaN,5250,2020
3,2019-12-31,2359399,54059,0.047,NaN,5250,2019
4,2018-12-31,2216099,51307,0.046,NaN,5250,2018
...,...,...,...,...,...,...,...
5212,2022-11-30,136167,42487,0.083,2.04,5131,2022
5213,2021-11-30,149616,45360,0.099,1.72,5131,2021
5214,2020-11-30,170678,46773,0.102,1.67,5131,2020
5215,2019-11-30,170622,49522,0.108,1.11,5131,2019


In [16]:
func={"annual_revenue('000)":'sum',"annual_net_income('000)":'sum',"annual_eps":'sum',"annual_dp(%)":'sum'}
annual_div_info=df2.groupby(['annual_stock_code','div_year']).agg(func)
annual_div_info.head(10)

annual_revenue('000)  annual_net_income('000)  \
annual_stock_code div_year                                                  
0001              2018                     88288                    12173   
                  2019                    122969                    18819   
                  2020                    128411                    22586   
                  2021                    145441                    25291   
                  2022                    120980                    26910   
0002              2018                    178476                    15749   
                  2019                    172550                    22195   
                  2020                    171727                    29557   
                  2021                    159622                    24390   
                  2022                    207917                    62095   

                            annual_eps  annual_dp(%)  
annual_stock_code div_year                            
0001              2018           0.021          0.00  
                  2019           0.029          0.51  
                  2020           0.034          0.44  
                  2021           0.034          0.44  
                  2022           0.035          0.57  
0002              2018           0.118          0.00  
                  2019           0.161          0.00  
                  2020           0.208          0.00  
                  2021           0.165          0.00  
                  2022           0.420          0.23

In [17]:
annual_div_info= annual_div_info.reset_index()

In [18]:
annual_div_info

,annual_stock_code,div_year,annual_revenue('000),annual_net_income('000),annual_eps,annual_dp(%)
0,0001,2018,88288,12173,0.021,0.00
1,0001,2019,122969,18819,0.029,0.51
2,0001,2020,128411,22586,0.034,0.44
3,0001,2021,145441,25291,0.034,0.44
4,0001,2022,120980,26910,0.035,0.57
...,...,...,...,...,...,...
5105,9997,2019,327539,566,0.004,0.00
5106,9997,2020,280178,3050,0.024,0.00
5107,9997,2021,339159,16317,0.126,0.18
5108,9997,2022,325005,14469,0.115,0.11


In [19]:
annual_div_info.to_csv("div_annual_report(Final).csv")